# **Apache Spark DataFrames Project**





**Project Deliverable**

You will be required to submit:

● A GitHub repository with your project written in Pyspark.

**Instructions**

As a Data professional, you need to perform an analysis by answering questions about some stock market data on Safaricom from the years 2012-2017.

You will need to perform the following:

**Data Importation and Exploration**

● Start a spark session and load the stock file while inferring the data types.
● Determine the column names
● Make observations about the schema.
● Show the first 5 rows
● Use the describe method to learn about the data frame
Data Preparation
● Format all the data to 2 decimal places i.e. format_number()
● Create a new data frame with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day

**Data Analysis**

● What day had the Peak High in Price?
● What is the mean of the Close column?
● What is the max and min of the Volume column?
● How many days was the Close lower than 60 dollars?
● What percentage of the time was the High greater than 80 dollars?
● What is the Pearson correlation between High and Volume?
● What is the max High per year?
● What is the average Close for each Calendar Month?

In [1]:
# Installing pyspark
# ---
#
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 43.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=eefb106d6aacb5be84f05bb4db89e6b00fcca9eeb4c8a82222a67c89f46ea94a
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
# Run a local spark session

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

 Register the DataFrame as a Table

In [8]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)
df_saf = sqlCtx.read.csv("saf_stock.csv")
df_saf.createOrReplaceTempView('saf_stock')
tables = sqlCtx.tableNames()
print(tables)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


['saf_stock']


In [11]:
print(type(df_saf))

<class 'pyspark.sql.dataframe.DataFrame'>


In [10]:
df_saf.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|       _c0|               _c1|      _c2|      _c3|               _c4|     _c5|               _c6|
+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [13]:
#Determine the column names
df_saf = sqlCtx.read.csv("saf_stock.csv", header=True)
print(df_saf.columns)

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']


In [15]:
df_saf.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [17]:
#describe method to learn about the data frame

df_saf.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|         10010500|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

In [20]:
#Format all the data to 2 decimal places i.e. format_number()

from pyspark.sql.functions import format_number

df_saf2=df_saf.select(df_saf['Date'],format_number(df_saf['Open'].cast('float'),2).alias('Open'),
              format_number(df_saf['High'].cast('float'), 2).alias('High'),
              format_number(df_saf['Low'].cast('float'), 2).alias('Low'),
              format_number(df_saf['Close'].cast('float'), 2).alias('Close'),
              df_saf['Volume'].cast('int').alias('Volume'),
              format_number(df_saf['Adj Close'].cast('float'), 2).alias('Adj Close')
              )

df_saf2.show(5)

+----------+-----+-----+-----+-----+--------+---------+
|      Date| Open| High|  Low|Close|  Volume|Adj Close|
+----------+-----+-----+-----+-----+--------+---------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|    52.62|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|    52.08|
|2012-01-05|59.35|59.62|58.37|59.42|12768200|    51.83|
|2012-01-06|59.42|59.45|58.87|59.00| 8069400|    51.46|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|    51.62|
+----------+-----+-----+-----+-----+--------+---------+
only showing top 5 rows



In [21]:
#Create a new data frame with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day

df_saf_HV = df_saf2.withColumn("HV Ratio",df_saf2['High']/df_saf2['Volume'])
print(df_saf_HV.show(5))


+----------+-----+-----+-----+-----+--------+---------+--------------------+
|      Date| Open| High|  Low|Close|  Volume|Adj Close|            HV Ratio|
+----------+-----+-----+-----+-----+--------+---------+--------------------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|    52.62|4.819714574387472E-6|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|    52.08|6.290848821573389...|
|2012-01-05|59.35|59.62|58.37|59.42|12768200|    51.83|4.669413073103491E-6|
|2012-01-06|59.42|59.45|58.87|59.00| 8069400|    51.46|7.367338339901356E-6|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|    51.62|8.915604928660188E-6|
+----------+-----+-----+-----+-----+--------+---------+--------------------+
only showing top 5 rows

None


**Data Analysis**

In [23]:
# What day had the Peak High in Price?

print(df_saf_HV.orderBy(df_saf_HV['High'].desc()).head(1)[0][0])

2015-01-13


In [24]:
#What is the mean of the Close column?

from pyspark.sql.functions import mean, max, min

print(df_saf_HV.select(mean('Close')).show())

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844992050863|
+-----------------+

None


In [25]:
#What is the max and min of the Volume column?

print(df_saf_HV.select(min('Volume'),max('Volume')).show())

+-----------+-----------+
|min(Volume)|max(Volume)|
+-----------+-----------+
|    2094900|   80898100|
+-----------+-----------+

None


In [26]:
# How many days was the Close lower than 60 dollars?

print(df_saf_HV.filter(df_saf_HV['Close'] < 60).count())

81


In [27]:
# What percentage of the time was the High greater than 80 dollars?

print((df_saf_HV.filter(df_saf_HV['High']>80).count()/df.count()) * 100)

8.419380460683081


In [28]:
# What is the Pearson correlation between High and Volume?

from pyspark.sql.functions import corr

print(df_saf_HV.select(corr('High','Volume')).show())


+--------------------+
|  corr(High, Volume)|
+--------------------+
|-0.33843260582148915|
+--------------------+

None


In [29]:
# What is the max High per year?

from pyspark.sql import functions as F
df_saf_HV.groupby(F.date_format('Date','yyyy').alias('Year')).agg({'High': 'max'}).sort('Year').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2012|    77.60|
|2013|    81.37|
|2014|    88.09|
|2015|    90.97|
|2016|    75.19|
+----+---------+



In [30]:
# What is the average Close for each Calendar Month?

df_saf_HV.groupby(F.date_format('Date','MM').alias('Month')).agg({'Close': 'mean'}).sort('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|   01|71.44801980198022|
|   02|71.30680412371134|
|   03|71.77794392523363|
|   04|72.97361904761907|
|   05|72.30971698113206|
|   06|72.49537735849057|
|   07|74.43971962616824|
|   08|73.02981818181819|
|   09|72.18411764705883|
|   10|71.57854545454546|
|   11|72.11108910891085|
|   12|72.84792452830189|
+-----+-----------------+

